<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="-unSkript-Runbooks-">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong>To delete unattached EBS volume using unSkript actions.</strong></div>
</center><center>
<h2 id="Delete-Unattached-EBS-Volume">Delete Unattached EBS Volume</h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener">Filter AWS Unattached EBS Volume</a><br>2)<a href="#2" target="_self" rel="noopener">Create Snapshot Of EBS Volume</a><br>3)<a href="#3" target="_self" rel="noopener">Delete EBS Volume</a></p>

In [2]:
if ebs_volume and not region:
    raise SystemExit("Provide region for the EBS Volumes!")

<h3 id="AWS-List-All-Regions">AWS List All Regions</h3>
<p>In this action, we list all the available regions from AWS if the user does not provide a region as a parameter.</p>
<blockquote>
<p>Output variable: <code>region</code></p>
</blockquote>

In [27]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')

task.configure(outputName="region")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3 id="Filter-Unattached-EBS-Volumes">Filter Unattached EBS Volumes</h3>
<p>Here we will use unSkript <strong>Filter AWS Unattached EBS Volume</strong> action. This action filters all the EBS volumes from the given region and returns a list of all the unattached EBS volumes. It will execute if the <code>ebs_volume</code> parameter is not passed.</p>
<blockquote>
<p>Input parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>unattached_volumes</code></p>
</blockquote>

In [28]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional
from unskript.legos.utils import CheckOutput, CheckOutputStatus
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint


from beartype import beartype
@beartype
def aws_filter_ebs_unattached_volumes_printer(output):
    if output is None:
        return
    pprint.pprint(output.json())


@beartype
def aws_filter_ebs_unattached_volumes(handle, region: str = "") -> CheckOutput:
    """aws_filter_ebs_unattached_volumes Returns an array of ebs volumes.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :rtype: CheckOutput with status result and list of EBS Unattached Volume.
    """
    result=[]
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            # Filtering the volume by region
            ec2Client = handle.resource('ec2', region_name=reg)
            volumes = ec2Client.volumes.all()

            # collecting the volumes which has zero attachments
            for volume in volumes:
                volume_dict = {}
                if len(volume.attachments) == 0:
                    volume_dict["region"] = reg
                    volume_dict["volume_id"] = volume.id
                    result.append(volume_dict)
        except Exception as e:
            pass

    return CheckOutput(status=CheckOutputStatus.SUCCESS,
                       objects=result,
                       error=str(""))


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not ebs_volume",
    "condition_result": true
    }''')
task.configure(outputName="unattached_volumes")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_ebs_unattached_volumes, lego_printer=aws_filter_ebs_unattached_volumes_printer, hdl=hdl, args=args)

<h3 id="Modify-Listener-ARNs-Output">Modify Unattached EBS Volume Output</h3>
<p>In this action, we modify the output from step 1 and return a list of dictionary items for the unattached EBS volume.</p>
<blockquote>
<p>Output variable: ebs_list</p>
</blockquote>

In [29]:
ebs_list = []
try:
    for k, v in unattached_volumes.items():
        vol = json.loads(v.json())
        if vol["status"] == 1:
            for volume in vol["objects"]:
                ebs_list.append(volume)
except Exception as e:
    for i in ebs_volume:
        data_dict = {}
        data_dict["region"] = region[0]
        data_dict["volume_id"] = i
        ebs_list.append(data_dict)

<h3 id="Create-Snapshot-Of-EBS-Volume">Create a Snapshot Of EBS Volume</h3>
<p>Here we will use the unSkript&nbsp;<strong>Create Snapshot Of EBS Volume</strong> action. In this action, we will back up the data stored in EBS volumes by passing the list of unattached EBS volumes from step 1 and creating a snapshot of the EBS volume of the EC2 instance.</p>
<blockquote>
<p>Input parameters: <code>volume_id</code>, <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>snapshot_metadata</code></p>
</blockquote>

In [30]:
##
##  Copyright (c) 2022 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_create_volumes_snapshot_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_create_volumes_snapshot(handle, volume_id: str, region: str) -> List:
    """aws_create_volumes_snapshot Returns an list containing SnapshotId.

        :type region: string
        :param region: used to filter the volume for a given region.

        :type volume_id: string
        :param volume_id: Volume ID to create snapshot for particular volume.

        :rtype: List containing SnapshotId.
    """
    result = []

    ec2Client = handle.resource('ec2', region_name=region)

    try:
        response = ec2Client.create_snapshot(VolumeId=volume_id)
        result.append(response)
    except Exception as e:
        raise e

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "volume_id": "iter.get(\\"volume_id\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "ebs_list",
    "iter_parameter": ["volume_id","region"]
    }''')

task.configure(outputName="snapshot_metadata")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_volumes_snapshot, lego_printer=aws_create_volumes_snapshot_printer, hdl=hdl, args=args)

<h3 id="Modify-Listener-ARNs-Output">Modify Snapshot Action Output</h3>
<p>In this action, we modify the output from step 2 and return a list of dictionary items for the volumes whose snapshot has been created.</p>
<blockquote>
<p>Output variable: snapshot_volumes</p>
</blockquote>

In [32]:
import json

snapshot_volumes = []
for k, v in snapshot_metadata.items():
    try:
        if v[0].id:
            snap_dict = json.loads(k.replace("\'", "\""))
            snapshot_volumes.append(snap_dict)
    except Exception as e:
        pass

<h3 id="Delete-EBS-Volume-by-Volume-ID">Delete EBS Volume by Volume ID</h3>
<p>In this action, we delete the unattached EBS volume we get after steps 1 and 2.</p>
<blockquote>
<p>Input parameters: <code>volume_id</code>, <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>deletion_information</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2022 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_delete_volumes_printer(output):
    if output is None:
        return
    pprint.pprint({"Output": output})


@beartype
def aws_delete_volumes(handle, volume_id: str, region: str) -> str:
    """aws_filter_ebs_unattached_volumes Returns an array of ebs volumes.

        :type handle: object
        :param handle: Object returned by the task.validate(...) method.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :type volume_id: string
        :param volume_id: Volume ID needed to delete particular volume.

        :rtype: Result of the API in the List form.
    """
    result = []

    ec2Client = handle.client('ec2',region_name=region)

    # Adding logic for deletion criteria
    try:
        response = ec2Client.delete_volume(VolumeId=volume_id,)
        result.append(response)
    except Exception as e:
        result.append(e)

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "volume_id": "iter.get(\\"volume_id\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "snapshot_volumes",
    "iter_parameter": ["volume_id","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(snapshot_volumes) > 0",
    "condition_result": true
    }''')

task.configure(outputName="deletion_information")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_volumes, lego_printer=aws_delete_volumes_printer, hdl=hdl, args=args)

### Conclusion
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions to filter unattached EBS volumes and create snapshots of those and delete them. To view the full platform capabunscriptedof unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>